In [1]:
import ROOT as r
import numpy as np

Welcome to JupyROOT 6.22/06


Definition of essential functions.

In [2]:
def thetaPhi():
    '''returns random theta and phi values'''
    theta = np.random.uniform(0,2*np.pi)
    phi = np.random.uniform(0,np.pi)
    return theta, phi

def momenta(mag, theta, phi):
    '''given an input magnitude, theta and phi the cartesian momenta are returned'''
    Px = mag * np.sin(theta) * np.cos(phi) 
    Py = mag * np.sin(theta) * np.sin(phi)
    Pz = mag * np.cos(theta)
    
    return Px, Py, Pz
    

def wArr(num):
    '''just for making numpy array of stationary w given the number of events'''
    momenta = np.zeros([num,4])
    momenta[:,3] = wMass
    
    return momenta

def vecToArr(vec):
    '''converts lorentzvector to array of components'''
    arr = np.zeros([4])
    arr[0] = vec.Px()
    arr[1] = vec.Py()
    arr[2] = vec.Pz()
    arr[3] = vec.E()
    
    return arr

In [3]:
def backToBack():
    ''' produces numpy array of 4 momenta of
        w pair with one hadronic decay and one leptonic decay
    '''
    event = np.zeros([6,4])
    # making back to back w bosons (stationary)
    wMass = 80.3
    
    wmag = 10# magnitude of w momentum
    
    theta, phi = thetaPhi()
    Px, Py, Pz = momenta(wmag, theta, phi)
    #print(Px, Py, Pz)
    
    m = Px**2 + Py**2 + Pz**2
    E = np.sqrt(m + wMass**2)

    w1 = r.Math.LorentzVector('ROOT::Math::PxPyPzE4D<double>')(Px, Py, Pz, E)
    w2 = r.Math.LorentzVector('ROOT::Math::PxPyPzE4D<double>')(-Px, -Py, -Pz, E)

    #boosting the Ws
    beta = np.random.normal(0,0.1)
    boostedW1 = r.Math.VectorUtil.boostZ(w1, beta)
    w1Arr = vecToArr(boostedW1)
    boostedW2 = r.Math.VectorUtil.boostZ(w2, beta)
    w2Arr = vecToArr(boostedW2)
    
    # w1Arr = vecToArr(w1)
    # w2Arr = vecToArr(w2)
    # boost1 = w1.BoostToCM()
    # boost2 = w2.BoostToCM()
    # have to make them into tlorentzvectors for the boostvector function to work

    boow1 = r.TLorentzVector()
    boow1.SetPxPyPzE(boostedW1.Px(), boostedW1.Py(), boostedW1.Pz(), boostedW1.E())
    boow2 = r.TLorentzVector()
    boow2.SetPxPyPzE(boostedW2.Px(), boostedW2.Py(), boostedW2.Pz(), boostedW2.E())

    # making boost vectors for the daughter particles of each w
    boost1 = boow1.BoostVector()
    boost2 = boow2.BoostVector()

    # boost1=  TVector3()
    # boost2=  TVector3()
    # boost1.SetXYZ(boostedW1.Px()/ boostedW1.E(), boostedW1.Py()/ boostedW1.E(), boostedW1.Pz()/ boostedW1.E())
    # boost2.SetXYZ(0,0,0.9)
    
    

    #making the hadronic decay particles this will use w1 as its parent
    #in w1 rest frame to start
    ej = wMass/2
    thetaj, phij = thetaPhi()
    Px, Py, Pz = momenta(ej, thetaj, phij)
    j1 = r.Math.LorentzVector('ROOT::Math::PxPyPzE4D<double>')(Px, Py, Pz, ej)
    j2 = -j1#Math.LorentzVector('ROOT::Math::PxPyPzE4D<double>')(-Px, -Py, -Pz, ej)
    j2.SetE(ej)

    # boosting jets to frame of W1
    j1Boost = r.Math.VectorUtil.boost(j1, boost1)
    j1Arr = vecToArr(j1Boost)
    j2Boost = r.Math.VectorUtil.boost(j2, boost1)
    j2Arr = vecToArr(j2Boost)

    # making leptonic decay particles using W2 as parent
    # w2 rest frame to start
    eE = wMass / 2
    thetae, phie = thetaPhi()
    Px, Py, Pz = momenta(eE, thetae, phie)
    e = r.Math.LorentzVector('ROOT::Math::PxPyPzE4D<double>')(Px, Py, Pz, eE)
    n = r.Math.LorentzVector('ROOT::Math::PxPyPzE4D<double>')(-Px, -Py, -Pz, eE)

    #boosting e and n to frame of W2
    eBoost = r.Math.VectorUtil.boost(e, boost2)
    eArr = vecToArr(eBoost)
    nBoost = r.Math.VectorUtil.boost(n, boost2)
    nArr = vecToArr(nBoost)

    #putting all particles into one array
    event[0] = w1Arr
    event[1] = w2Arr
    event[2] = j1Arr
    event[3] = j2Arr
    event[4] = eArr
    event[5] = nArr
    
    return event, beta

Creating arrays with stored data.

In [4]:
# looping this function can give big array of data
num = 100
events = np.zeros([num, 6, 4])
beta = np.zeros([num])
mag = np.zeros([num])

for i in range(num):
    events[i], beta[i]= backToBack()

Creating TTree with more branches to accommodate two W bosons, two jets, electron and beta value.

In [6]:
tree = r.TTree("tree", "initial random e and n components")

w1Branch = np.zeros(4)
w2Branch = np.zeros(4)
jet1Branch = np.zeros(4)
jet2Branch = np.zeros(4)
eBranch = np.zeros(4)
boostValue = np.zeros(4)

tree.Branch("w1Boson",w1Branch,"w1Array[4]/D")
tree.Branch("w2Boson",w2Branch,"w2Array[4]/D")
tree.Branch("jet1",jet1Branch,"jet1Array[4]/D")
tree.Branch("jet2",jet2Branch,"jet2Array[4]/D")
tree.Branch("electron",eBranch,"elArray[4]/D")
tree.Branch("beta", boostValue,"boostArray[4]/D")

In [7]:
# Filling values into TTree
for i in range(num):
    for j in range(4):
        w1Branch[j] = events[i,0,j]
        w2Branch[j] = events[i,1,j]
        jet1Branch[j] = events[i,2,j]
        jet2Branch[j] = events[i,3,j]
        eBranch[j] = events[i,4,j]
    
    boostValue[0] = beta[i]
    
    tree.Fill()

By scanning the TTree, we can see its structure.

In [8]:
tree.Scan()

400

***********************************************************************************************
*    Row   * Instance * w1Boson.w * w2Boson.w * jet1.jet1 * jet2.jet2 * electron. * beta.beta *
***********************************************************************************************
*        0 *        0 * -9.093357 * 9.0933577 * -39.67839 * 30.585034 * -11.33972 * -0.100287 *
*        0 *        1 * -2.005479 * 2.0054793 * -18.17224 * 16.166761 * 10.171273 *         0 *
*        0 *        2 * -11.82030 * -4.492571 * -16.76515 * 4.9448492 * -37.96411 *         0 *
*        0 *        3 * 81.697739 * 80.962857 * 46.751208 * 34.946531 * 40.906210 *         0 *
*        1 *        0 * -2.739752 * 2.7397526 * -11.40793 * 8.6681785 * 37.221931 * 0.0860848 *
*        1 *        1 * 7.8794071 * -7.879407 * 28.564535 * -20.68512 * -2.388246 *         0 *
*        1 *        2 * 1.4570068 * 12.526916 * -29.43297 * 30.889979 * 24.682914 *         0 *
*        1 *        3 * 80.745304 * 81.6

Type <CR> to continue or q to quit ==> Type <CR> to continue or q to quit ==> Type <CR> to continue or q to quit ==> Type <CR> to continue or q to quit ==> Type <CR> to continue or q to quit ==> Type <CR> to continue or q to quit ==> Type <CR> to continue or q to quit ==> Type <CR> to continue or q to quit ==> Type <CR> to continue or q to quit ==> Type <CR> to continue or q to quit ==> Type <CR> to continue or q to quit ==> Type <CR> to continue or q to quit ==> Type <CR> to continue or q to quit ==> Type <CR> to continue or q to quit ==> Type <CR> to continue or q to quit ==> Type <CR> to continue or q to quit ==> 

In [10]:
file = r.TFile("events2W.root","recreate")
tree.Write()
file.Close